In [2]:
!pwd

/home/cloudera


In [3]:
!mkdir DAMLASB

mkdir: cannot create directory `DAMLASB': File exists


In [4]:
cd DAMLASB/

/home/cloudera/DAMLASB


In [5]:
!pwd

/home/cloudera/DAMLASB


In [6]:
!ls -ltr

total 4
drwxrwxr-x 2 cloudera cloudera 4096 Aug 30 02:39 map_reduce


In [7]:
!mkdir map_reduce

mkdir: cannot create directory `map_reduce': File exists


In [8]:
cd map_reduce/

/home/cloudera/DAMLASB/map_reduce


In [9]:
!ls -ltr

total 0


In [10]:
!pip install mrjob

    100% |████████████████████████████████| 286kB 707kB/s 
    100% |████████████████████████████████| 51kB 4.0MB/s 
    100% |████████████████████████████████| 215kB 307kB/s 
    100% |████████████████████████████████| 81kB 75kB/s 
    100% |████████████████████████████████| 81kB 99kB/s 
    100% |████████████████████████████████| 51kB 1.9MB/s 
  Running setup.py bdist_wheel for filechunkio ... - \ done
  Stored in directory: /home/cloudera/.cache/pip/wheels/26/ae/6a/d93b918a6a07a0afab4b1c51681047657d1300160ab7045565
  Running setup.py bdist_wheel for httplib2 ... - \ done
  Stored in directory: /home/cloudera/.cache/pip/wheels/c7/67/60/e0be8ccfc1e08f8ff1f50d99ea5378e204580ea77b0169fb55
  Running setup.py bdist_wheel for uritemplate ... - \ done
  Stored in directory: /home/cloudera/.cache/pip/wheels/2d/dc/57/124fcb62028d04cf74f6c7f5261f5deb29b3f6022eec179064
  Running setup.py bdist_wheel for oauth2client ... - \ done
  Stored in directory: /home/cloudera/.cache/pip/w

In [42]:
!export PATH="/home/cloudera/anaconda/bin:$PATH"

In [11]:
!ls -ltr

total 0


In [12]:
! find / -name "hadoop*streaming*" -print 1> tmp.hadoopJar 2> /dev/null

In [13]:
!cat tmp.hadoopJar

/usr/share/doc/hadoop-2.6.0+cdh5.4.2+567/hadoop-streaming
/usr/lib/oozie/oozie-sharelib-mr1/lib/mapreduce-streaming/hadoop-streaming.jar
/usr/lib/oozie/oozie-sharelib-yarn/lib/mapreduce-streaming/hadoop-streaming.jar
/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.4.2.jar
/usr/lib/hadoop-mapreduce/hadoop-streaming.jar
/usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar
/usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-mr1.jar
/usr/jars/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar
/usr/jars/hadoop-streaming-2.6.0-cdh5.4.2.jar


In [15]:
!ls -l /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar

lrwxrwxrwx 1 root root 56 Jun  9  2015 /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar -> ../../../../jars/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar


In [16]:
!mkdir WordCount

In [17]:
!ls -ltr

total 8
-rw-rw-r-- 1 cloudera cloudera  585 Aug 30 07:29 tmp.hadoopJar
drwxrwxr-x 2 cloudera cloudera 4096 Aug 30 07:32 WordCount


In [18]:
%%writefile WordCount/word_count_mapper.py
#!/usr/bin/env python


import sys
import re
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

for line in sys.stdin:
    line= re.findall(r'[a-z]+', line.lower())
    line=" ".join(line)
    words = line.split()
    for word in words:
        print '%s\t%s' % (word, 1)


Writing WordCount/word_count_mapper.py


In [19]:
!ls -ltr

total 8
-rw-rw-r-- 1 cloudera cloudera  585 Aug 30 07:29 tmp.hadoopJar
drwxrwxr-x 2 cloudera cloudera 4096 Aug 30 07:48 WordCount


In [20]:
!ls WordCount/

word_count_mapper.py


In [22]:
!cat WordCount/word_count_mapper.py

#!/usr/bin/env python


import sys
import re
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

for line in sys.stdin:
    line= re.findall(r'[a-z]+', line.lower())
    line=" ".join(line)
    words = line.split()
    for word in words:
        print '%s\t%s' % (word, 1)

In [23]:
%%writefile WordCount/word_count_reducer.py
#!/usr/bin/env python

import sys
import re
cur_key = None
cur_count = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)

Writing WordCount/word_count_reducer.py


In [24]:
!chmod a+x WordCount/word_count_mapper.py

In [25]:
!chmod a+x WordCount/word_count_reducer.py

In [44]:
!echo "foo foo quux labs foo bar quux" | WordCount/word_count_mapper.py 

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are  you
foo	1
foo	1
quux	1
labs	1
foo	1
bar	1
quux	1


In [45]:
!echo "foo foo quux labs foo bar quux" | WordCount/word_count_mapper.py | sort -k1,1 | WordCount/word_count_reducer.py

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are  you
reporter:counter:Reducer Counters,Calls,1
bar	1
foo	3
labs	1
quux	2


In [27]:
!hdfs dfs -ls

In [28]:
!hdfs dfs -pwd

-pwd: Unknown command


In [29]:
!curl 'http://www.gutenberg.org/cache/epub/28885/pg28885.txt' -o alice_input_file.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  173k  100  173k    0     0  76590      0  0:00:02  0:00:02 --:--:-- 87446


In [30]:
!ls -ltr

total 184
-rw-rw-r-- 1 cloudera cloudera    585 Aug 30 07:29 tmp.hadoopJar
drwxrwxr-x 2 cloudera cloudera   4096 Aug 30 08:03 WordCount
-rw-rw-r-- 1 cloudera cloudera 177428 Aug 30 08:18 alice_input_file.txt


In [31]:
!hdfs dfs -copyFromLocal alice_input_file.txt

In [32]:
!hdfs dfs -ls

Found 1 items
-rw-r--r--   1 cloudera cloudera     177428 2016-08-30 08:21 alice_input_file.txt


In [38]:
!hdfs dfs -rm -r wordcount-output

16/08/30 08:30:10 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted wordcount-output


In [151]:
!pwd

/home/cloudera/DAMLASB/map_reduce


In [153]:
!hdfs dfs -rm -r wordcount-output
!hadoop jar /usr/jars/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar \
   -mapper /home/cloudera/DAMLASB/map_reduce/WordCount/word_count_mapper.py \
   -reducer /home/cloudera/DAMLASB/map_reduce/WordCount/word_count_reducer.py \
   -combiner /home/cloudera/DAMLASB/map_reduce/WordCount/word_count_reducer.py \
   -input alice_input_file.txt \
   -output wordcount-output  \
   -numReduceTasks 3

16/08/30 21:55:26 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted wordcount-output
packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.4.2.jar] /tmp/streamjob3707439227408718973.jar tmpDir=null
16/08/30 21:55:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/30 21:55:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/30 21:55:31 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/30 21:55:31 INFO mapreduce.JobSubmitter: number of splits:2
16/08/30 21:55:32 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472567065777_0008
16/08/30 21:55:32 INFO impl.YarnClientImpl: Submitted application application_1472567065777_0008
16/08/30 21:55:32 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1472567065777_0008/
16/08/30 21:55:32 INFO mapreduce.Job: Running job: job_1472567065777_0008
16

## HW-2.1: Find longest word in a text

In [51]:
!mkdir longest_word

In [52]:
%%writefile longest_word/mapper.py
#!/usr/bin/env python


import sys
import re
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

for line in sys.stdin:
    line= re.findall(r'[a-z]+', line.lower())
    line=" ".join(line)
    words = line.split()
    for word in words:
        val=len(word)
        print '%s\t%s' % (word, val)


Writing longest_word/mapper.py


In [67]:
%%writefile longest_word/reducer.py
#!/usr/bin/env python

import sys
import re
cur_key = None
cur_length = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_length = int(value)
    else:
        if cur_key:
            if cur_length > int(value):
                cur_key=cur_key
                cur_length=cur_length
            else:
                cur_key = key
                cur_length = int(value)
        else:
            cur_key = key
            cur_length = int(value)
            
print '%s\t%s' % (cur_key, cur_length)

Overwriting longest_word/reducer.py


In [68]:
!chmod a+x longest_word/mapper.py
!chmod a+x longest_word/reducer.py

In [70]:
!echo "foo foo quux labs foo bar quux aaaaaaaa lnsdfflkjbelfbd" | longest_word/mapper.py | sort -k1,1 | longest_word/reducer.py

reporter:counter:Reducer Counters,Calls,1
reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are  you
lnsdfflkjbelfbd	15


In [106]:
!cat alice_input_file.txt | longest_word/mapper.py | sort -k1,1 | longest_word/reducer.py

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are  you
reporter:counter:Reducer Counters,Calls,1
unenforceability	16


In [156]:
!hdfs dfs -rm -r longest_word-output
!hdfs dfs -mkdir longest_word
!hadoop jar /usr/jars/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar \
   -mapper /home/cloudera/DAMLASB/map_reduce/longest_word/mapper.py \
   -reducer /home/cloudera/DAMLASB/map_reduce/longest_word/reducer.py \
   -combiner /home/cloudera/DAMLASB/map_reduce/longest_word/reducer.py \
   -input alice_input_file.txt \
   -output longest_word-output  \
   -numReduceTasks 3

rm: `longest_word-output': No such file or directory
packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.4.2.jar] /tmp/streamjob5772638991259276456.jar tmpDir=null
16/08/30 22:12:02 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/30 22:12:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/30 22:12:03 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/30 22:12:04 INFO mapreduce.JobSubmitter: number of splits:2
16/08/30 22:12:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472567065777_0009
16/08/30 22:12:04 INFO impl.YarnClientImpl: Submitted application application_1472567065777_0009
16/08/30 22:12:04 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1472567065777_0009/
16/08/30 22:12:04 INFO mapreduce.Job: Running job: job_1472567065777_0009
16/08/30 22:12:13 INFO mapreduce.Job: Job job_1472567065777_0009 running in uber mode : false
16/08/30 22:12:13

In [157]:
!hdfs dfs -cat longest_word-output/part-0000*

unenforceability	16
representations	15
redistributing	14


## HW-2.2: Number of uppercase words and lower case words

In [71]:
!mkdir upper_lower

In [183]:
%%writefile upper_lower/mapper.py
#!/usr/bin/env python


import sys
import re
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

for line in sys.stdin:
    line= re.findall(r'[a-zA-Z]+', line)
    line=" ".join(line)
    words = line.split()
    for word in words:
        if word[0].isupper()==True:
            case='uppercase'
            print '%s\t%s\t%s' % (word,1,case)
        else:
            case='lowercase'
            print '%s\t%s\t%s' % (word,1,case)


Overwriting upper_lower/mapper.py


In [214]:
%%writefile upper_lower/reducer.py
#!/usr/bin/env python

import sys
import re
upper_key = None
lower_key = None
upper_count = 0
lower_count = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value, case = line.split()
    if case =='uppercase':
        if key == upper_key:
            upper_count = upper_count
        else:
            if upper_key:
                upper_key = key
                upper_count = upper_count+1
            else:
                upper_key = key
                upper_count = int(value)
    else:
        if key == lower_key:
            lower_count = lower_count
        else:
            if lower_key:
                lower_key = key
                lower_count = lower_count+1
            else:
                lower_key = key
                lower_count = int(value)

print '%s\t%s' % ('lowercase_words', lower_count)
print '%s\t%s' % ('uppercase_words', upper_count)

Overwriting upper_lower/reducer.py


In [215]:
!chmod a+x upper_lower/mapper.py
!chmod a+x upper_lower/reducer.py

In [223]:
!cat alice_input_file.txt  | upper_lower/mapper.py | sort -k1,1 | upper_lower/reducer.py

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are  you
reporter:counter:Reducer Counters,Calls,1
lowercase_words	2882
uppercase_words	702


In [226]:
!hdfs dfs -rm -r upper_lower-output
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar \
   -mapper /home/cloudera/DAMLASB/map_reduce/upper_lower/mapper.py \
   -reducer /home/cloudera/DAMLASB/map_reduce/upper_lower/reducer.py \
   -input alice_input_file.txt \
   -output upper_lower-output  \
   -numReduceTasks 1

16/08/31 03:19:55 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted upper_lower-output
packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.4.2.jar] /tmp/streamjob315440929305937234.jar tmpDir=null
16/08/31 03:20:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 03:20:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 03:20:01 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/31 03:20:01 INFO mapreduce.JobSubmitter: number of splits:2
16/08/31 03:20:01 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472567065777_0029
16/08/31 03:20:02 INFO impl.YarnClientImpl: Submitted application application_1472567065777_0029
16/08/31 03:20:02 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1472567065777_0029/
16/08/31 03:20:02 INFO mapreduce.Job: Running job: job_1472567065777_0029
1

In [227]:
!hdfs dfs -cat upper_lower-output/part-0000*

lowercase_words	2882
uppercase_words	702


## HW-2.3: Lowest temperature at Heathrow

In [103]:
!curl 'http://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/heathrowdata.txt' -o uk_heathrow_input_file.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42517  100 42517    0     0  50470      0 --:--:-- --:--:-- --:--:-- 76332


In [104]:
!ls -ltr

total 236
-rw-rw-r-- 1 cloudera cloudera    585 Aug 30 07:29 tmp.hadoopJar
drwxrwxr-x 2 cloudera cloudera   4096 Aug 30 08:03 WordCount
-rw-rw-r-- 1 cloudera cloudera 177428 Aug 30 08:18 alice_input_file.txt
drwxrwxr-x 2 cloudera cloudera   4096 Aug 30 09:51 longest_word
drwxrwxr-x 2 cloudera cloudera   4096 Aug 30 11:53 upper_lower
-rw-rw-r-- 1 cloudera cloudera  42517 Aug 30 12:02 uk_heathrow_input_file.txt


In [107]:
!head uk_heathrow_input_file.txt

Heathrow (London Airport)
Location 507800E 176700N, Lat 51.479 Lon -0.449, 25m amsl
Estimated data is marked with a * after the value.
Missing data (more than 2 days missing in month) is marked by  ---.
Sunshine data taken from an automatic Kipp & Zonen sensor marked with a #, otherwise sunshine data taken from a Campbell Stokes recorder.
   yyyy  mm   tmax    tmin      af    rain     sun
              degC    degC    days      mm   hours
   1948   1    8.9     3.3    ---     85.0    ---
   1948   2    7.9     2.2    ---     26.0    ---
   1948   3   14.2     3.8    ---     14.0    ---


In [108]:
!mkdir lowest_temp

In [136]:
%%writefile lowest_temp/mapper.py
#!/usr/bin/env python


import sys
import re
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

for line in sys.stdin:
    line= re.findall(r'[0-9.]+', line)
    if line != []:
        #words = line.split()
        #line = line.strip()
        if (len(line[0]) == 4 and 1 <= int(line[1]) <= 12):
            print '%s\t%s' % (line[0], line[3])

Writing lowest_temp/mapper.py


In [167]:
%%writefile lowest_temp/reducer.py
#!/usr/bin/env python

import sys
import re
min_temp = 0.0
final_key = None
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == final_key:
        if float(value) > min_temp:
            min_temp = min_temp
        else:
            min_temp = value
    else:
        if final_key:
            if float(value) > min_temp:
                min_temp = min_temp
            else:
                min_temp = float(value)
                final_key = key
        else:
            final_key = key
            min_temp = float(value)

print '%s\t%s' % (final_key, min_temp)

Overwriting lowest_temp/reducer.py


In [168]:
!chmod a+x lowest_temp/mapper.py
!chmod a+x lowest_temp/reducer.py

In [169]:
!cat uk_heathrow_input_file.txt | lowest_temp/mapper.py | sort -k1,1 | lowest_temp/reducer.py

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are  you
reporter:counter:Reducer Counters,Calls,1
1963	0.0


In [149]:
!hdfs dfs -ls 
!hdfs dfs -copyFromLocal uk_heathrow_input_file.txt
!hdfs dfs -ls

Found 2 items
-rw-r--r--   1 cloudera cloudera     177428 2016-08-30 08:21 alice_input_file.txt
drwxr-xr-x   - cloudera cloudera          0 2016-08-30 09:49 wordcount-output
Found 3 items
-rw-r--r--   1 cloudera cloudera     177428 2016-08-30 08:21 alice_input_file.txt
-rw-r--r--   1 cloudera cloudera      42517 2016-08-30 21:36 uk_heathrow_input_file.txt
drwxr-xr-x   - cloudera cloudera          0 2016-08-30 09:49 wordcount-output


In [204]:
!hdfs dfs -rm -r lowest_temp-output
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.2.jar \
   -mapper /home/cloudera/DAMLASB/map_reduce/lowest_temp/mapper.py \
   -reducer /home/cloudera/DAMLASB/map_reduce/lowest_temp/reducer.py \
   -combiner /home/cloudera/DAMLASB/map_reduce/lowest_temp/reducer.py \
   -input uk_heathrow_input_file.txt \
   -output lowest_temp-output  \
   -numReduceTasks 1

16/08/31 02:41:54 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted lowest_temp-output
packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.4.2.jar] /tmp/streamjob8687711466530980004.jar tmpDir=null
16/08/31 02:41:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 02:41:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 02:41:59 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/31 02:41:59 INFO mapreduce.JobSubmitter: number of splits:2
16/08/31 02:41:59 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472567065777_0021
16/08/31 02:42:00 INFO impl.YarnClientImpl: Submitted application application_1472567065777_0021
16/08/31 02:42:00 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1472567065777_0021/
16/08/31 02:42:00 INFO mapreduce.Job: Running job: job_1472567065777_0021


In [205]:
!hdfs dfs -cat lowest_temp-output/part-0000*

1963	0.0


In [177]:
x ="hello"
y = "sir"
z = "2"
a = "%s\t%s %s" % (x,y,z)
print(a)
print(a.split())

hello	sir 2
['hello', 'sir', '2']


In [191]:
man hdfs